### Import libraries

In [ ]:
#import argparse
import ast
import itertools
import numpy as np
import os
import pandas as pd
import re
import requests
import time
import warnings
warnings.filterwarnings("ignore")

### Determine sessions to be processed

In [ ]:
path_to_repo = os.path.join("path/to/repo")

In [ ]:
logs = pd.read_csv(os.path.join(path_to_repo, 
                                "data", 
                                "processed-data", 
                                "experiment-logs.csv"))

In [ ]:
end_idx = 0
for index, row in logs.iterrows():
    lst = list(row)
    if all(item != item for item in lst):
        end_idx = index
        break
        
logs = logs[:end_idx]
logs.tail()

In [ ]:
lst = list(logs["Date"])
for i in range(1, len(lst)):
    if lst[i] != lst[i]:
        lst[i] = lst[i-1]
logs["DateFull"] = lst
logs.head()

In [ ]:
logs = logs[logs['Use?'] == 'Yes']

def get_filenames(row):
    date = row["DateFull"]
    run = row["Run number"]
    cond = "-".join(row["Condition"].split("_"))
    file_name = "-".join(["experiment", date, run, cond]) + ".csv"
    return file_name
    
logs["file_name"] = logs.apply(get_filenames, axis = 1)
logs.head()

In [ ]:
lst_filenames = list(logs["file_name"])

print(len(lst_filenames))
lst_filenames[:5]

### Define functions

##### (i) Convert each raw part into rectangular format

In [ ]:
def makeRectangular(raw_data):
    numTotalEvents = len(raw_data.data_name.values)
    numUniqEvents = len(set(raw_data.data_name.values))
    rect_data = pd.DataFrame(columns = raw_data.data_name.values[:numUniqEvents])
    start = 0
    end = numUniqEvents
    while start < numTotalEvents:
        cur = pd.DataFrame([list(raw_data.data_value[start:end])], 
                           columns = raw_data.data_name.values[start:end])
        # added sort = True to the next line after seeing warning message
        rect_data = rect_data.append(cur, ignore_index = True, sort = True)
        start = end
        end = end + numUniqEvents
    if 'round' in rect_data.columns:
        rect_data = rect_data[(rect_data['round'] != '0') | (rect_data['version'] == '2')]
    return rect_data

##### (ii) Restrict dataset to tutorial clicks

In [ ]:
def contains_tutorial(row):
    return "Tutorial" in row.event

### Process the data

In [ ]:
in_path = "in/path"
out_path = os.path.join(path_to_repo, "data", "processed-data")

In [ ]:
for file in lst_filenames:
    
    # Read raw csv data
    df_raw = pd.read_csv(os.path.join(in_path, file))

    # Store date and hour
    date = re.search('(.*?)T', df_raw['event_date'][0]).group(1)
    hour = re.search('T(.*?)Z', df_raw['event_date'][0]).group(1)

    # Remove sensitive and unnecessary rows
    remove = ["acceptLanguage", "acceptEncoding", "userAgent", "host", "requestURI", "referer", "connection"]
    df_raw = df_raw[~df_raw.data_name.isin(remove)]

    # Replace pid with anonymous identifiers
    pattern = re.compile("[A-Z0-9]{30}")
    pidsDic = {}
    idx = 1
    for index, row in df_raw.iterrows():
        if pattern.match(str(row.data_value)) and row.data_value not in pidsDic:
            pidsDic[str(row.data_value)] = "p" + str(idx)
            idx += 1
    for index, row in df_raw.iterrows():
        if pattern.findall(str(row.data_value)):
            for match in pattern.findall(str(row.data_value)):
                df_raw.loc[index, 'data_value'] = str(row.data_value).replace(match, pidsDic[match])
        if pattern.findall(str(row.data_name)):
            for match in pattern.findall(str(row.data_name)):
                df_raw.loc[index, 'data_name'] = str(row.data_name).replace(match, pidsDic[match])

    # Separate data into its different parts
    tutorials = df_raw[df_raw.apply(contains_tutorial, axis = 1)]
    tutorials = tutorials.pivot(index = 'data_value', 
                                columns = 'event', 
                                values = 'event_date').reset_index()
    tutorials = tutorials.rename(columns = {"data_value": "pid"})
    if 'Tutorial300' in tutorials.columns:
        tutorials['Tutorial301'] = np.nan
    else:
        tutorials['Tutorial300'] = np.nan
    tutorials = tutorials[['pid', 'Tutorial100', 'Tutorial101', 'Tutorial102', 'Tutorial103',
                                  'Tutorial104', 'Tutorial105', 'Tutorial106', 'Tutorial107',
                                  'Tutorial108', 'Tutorial109', 'Tutorial110', 'Tutorial200',
                                  'Tutorial201', 'Tutorial202', 'Tutorial203', 'Tutorial204',
                                  'Tutorial205', 'Tutorial206', 'Tutorial207', 'Tutorial208',
                                  'Tutorial209', 'Tutorial300', 'Tutorial301', 'Tutorial302', 
                                  'Tutorial303', 'Tutorial304']]

    ip = df_raw[(df_raw['data_name'] == 'clientId') | (df_raw['data_name'] == 'ipAddress')].reset_index()
    ip['clientId'] = np.nan
    ip['ipAddress'] = np.nan
    for i in range(0, len(ip)-1, 2):
        if ip['data_name'][i] == 'clientId':
            ip['clientId'][i] = ip['data_value'][i]
            ip['ipAddress'][i] = ip['data_value'][i+1]
        else:
            ip['clientId'][i] = ip['data_value'][i+1]
            ip['ipAddress'][i] = ip['data_value'][i]
    ip = ip.dropna().drop_duplicates(subset = 'clientId').drop(
        labels = ['index', 'event', 'data_name', 'data_value'], axis = 1).reset_index(drop = True)
    ip = ip.drop(['id', 'event_date'], axis=1)

    screening = df_raw[df_raw['event'] == 'Screening']
    gameParams1 = df_raw[df_raw['event'] == 'GameParameters1']
    gameParams2 = df_raw[df_raw['event'] == 'GameParameters2']
    words = df_raw[df_raw['event'] == 'Words']
    
    coopChoiceNeighbors = df_raw[df_raw['event'] == 'CoopChoice']
    coopChoice = coopChoiceNeighbors[~coopChoiceNeighbors['data_name'].str.contains('neighbor')]
    neighbors = coopChoiceNeighbors[coopChoiceNeighbors['data_name'].str.contains('neighbor')]
    neighbors = neighbors.sort_values(by = ['data_name'])
    
    addChoice = df_raw[df_raw['event'] == 'AddChoice']
    cutChoice = df_raw[df_raw['event'] == 'CutChoice']
    score2 = df_raw[df_raw['event'] == 'Score2']
    score3 = df_raw[df_raw['event'] == 'Score3']
    fairScore2 = df_raw[df_raw['event'] == 'FairScore2']
    fairScore3 = df_raw[df_raw['event'] == 'FairScore3']
    trustScore2 = df_raw[df_raw['event'] == 'TrustScore2']
    trustScore3 = df_raw[df_raw['event'] == 'TrustScore3']
    relative = df_raw[df_raw['event'] == 'Relative']
    whyCoop = df_raw[df_raw['event'] == 'WhyCoop']
    demographic = df_raw[df_raw['event'] == 'Demographic'].drop(['event'], axis = 1)

    # Drop unnecessary columns
    df_raw = df_raw.drop(['id', 'event_date'], axis = 1)
    
    # Combine session metadata
    df_session_metadata = pd.concat([
        makeRectangular(gameParams1), 
        makeRectangular(gameParams2),
        pd.DataFrame([date],columns = ['date']),
        pd.DataFrame([hour],columns = ['hour'])], axis = 1)

    # Combine player metadata
    if relative.empty:
        df_player_metadata = ip.rename(
            columns = {'clientId':'pid', 'ipAddress':'ip_to_country'}, inplace = False).merge(
            tutorials, on='pid', how='outer').merge(
            makeRectangular(screening), on='pid', how='outer').merge(
            makeRectangular(words), on='pid', how='outer').merge(
            makeRectangular(score2), on='pid', how='outer').merge(
            makeRectangular(score3), on='pid', how='outer').merge(
            makeRectangular(fairScore2), on='pid', how='outer').merge(
            makeRectangular(fairScore3), on='pid', how='outer').merge(
            makeRectangular(trustScore2), on='pid', how='outer').merge(
            makeRectangular(trustScore3), on='pid', how='outer').merge(
            makeRectangular(whyCoop), on='pid', how='outer').merge(
            makeRectangular(demographic), on='pid', how='outer')
    else:
        df_player_metadata = ip.rename(
            columns = {'clientId':'pid', 'ipAddress':'ip_to_country'}, inplace = False).merge(
            tutorials, on='pid', how='outer').merge(
            makeRectangular(screening), on='pid', how='outer').merge(
            makeRectangular(words), on='pid', how='outer').merge(
            makeRectangular(score2), on='pid', how='outer').merge(
            makeRectangular(score3), on='pid', how='outer').merge(
            makeRectangular(fairScore2), on='pid', how='outer').merge(
            makeRectangular(fairScore3), on='pid', how='outer').merge(
            makeRectangular(trustScore2), on='pid', how='outer').merge(
            makeRectangular(trustScore3), on='pid', how='outer').merge(
            makeRectangular(relative), on='pid', how='outer').merge(
            makeRectangular(whyCoop), on='pid', how='outer').merge(
            makeRectangular(demographic), on='pid', how='outer')

    # Replace IP address with country
    for index, row in df_player_metadata.iterrows():
        url = 'http://ipinfo.io/' + str(row.ip_to_country) + '?token=XXXXXXXXXXXXXX'
        response = requests.get(url)
        response_text = re.sub("true", "True", response.text)
        response_text = re.sub("false", "False", response_text)
        d = ast.literal_eval(response_text)
        if "country" in d.keys():
            df_player_metadata.loc[index, 'ip_to_country'] = d["country"]
        else:
            df_player_metadata.loc[index, 'ip_to_country'] = "unknown"
        time.sleep(.1)

    # Combine game data
    neighbors2 = neighbors['data_name'].str.split('_', n = 3, expand = True)
    neighbors2.columns = ['pid', 'version', 'round', 'neighbor_no']
    neighbors2['data_value'] = neighbors['data_value']
    neighbors2 = neighbors2[(neighbors2['round'] != '0') | (neighbors2['version'] == '2')].reset_index(drop = True)
    
    neighbors3 = neighbors2.copy()

    pattern1 = re.compile(r"neighbor_0")
    pattern2 = re.compile(r"(neighbor_[1-9]{1})|(neighbor_[0-9]{2})")

    for index, row in neighbors3.iterrows():
        if pattern1.match(str(row.neighbor_no)):
            next_row = index + 1
            while next_row < len(neighbors3) and pattern2.match(str(neighbors3.iloc[next_row].neighbor_no)):
                neighbors3.iloc[index].data_value = \
                    neighbors3.iloc[index].data_value + ", " + str(neighbors3.iloc[next_row].data_value)
                next_row += 1

    func = np.vectorize(lambda x: bool(pattern2.match(x)))

    neighbors3 = neighbors3[~func(neighbors3.neighbor_no)].replace("neighbor_0", "neighbors")
    neighbors3 = neighbors3.drop(columns = ['neighbor_no'])
    neighbors3 = neighbors3.rename(columns = {"data_value": "neighbors"})
    neighbors3['neighbors'] = "[" + neighbors3['neighbors'] + "]"
    
    df_game_data = makeRectangular(coopChoice).merge(
        makeRectangular(addChoice), on=['pid', 'round', 'version'], how='outer').merge(
        makeRectangular(cutChoice), on=['pid', 'round', 'version'], how='outer', suffixes=('_add', '_cut')).merge(
        neighbors3, on=['pid', 'round', 'version'], how='outer')

    # Combine everything together
    df_full = df_game_data.merge(df_player_metadata, on='pid')
    df_full = pd.concat([df_full, df_session_metadata], axis = 1)
    df_full.date.replace([float('nan')], [df_full.date[0]], inplace=True)
    df_full.hour.replace([float('nan')], [df_full.hour[0]], inplace=True)
    df_full.earned1.replace([float('nan')], [df_full.earned1[0]], inplace=True)
    df_full.equal1.replace([float('nan')], [df_full.equal1[0]], inplace=True)
    df_full.earned2.replace([float('nan')], [df_full.earned2[0]], inplace=True)
    df_full.equal2.replace([float('nan')], [df_full.equal2[0]], inplace=True)

    # Write to csv
    df_full.to_csv(os.path.join(out_path, "processed-" + str(file)), sep = ',')